In [1]:

from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components import CallChannel, ComponentStarter, numpy_sample_setup
from utils import mon_samples
from components import (
    TwoWheelsV3, LoggerComponent, 
    BlueToothCarControlSPP_V2, start_bluetooth_server_v2, 
    AngularSpeedControlV3, 
    Picamera2V2,
    ImageMLControllerV4
)
from components.syncronisation import get_switches
from data_collection.data_collection import LoggerSet
import numpy as np
import datetime
from multiprocessing import Manager

import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

# from components import numpy_sample_setup
# r, w = numpy_sample_setup(['d', 'd'], [0, 0])

%load_ext autoreload
%autoreload 2

In [2]:
loggerset = LoggerSet('../log/testsession'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

In [3]:
import tensorflow.keras as keras # type: ignore 


def get_model():
    model_path = './training/11Jun/29Jun-save.keras'
    ml_model = keras.models.load_model(model_path) 
    def model(arr, values: List[Dict]):
        if len(values):
            #np.stack([v['acc'] for v in values], axis=-1).mean(-1) # the frames only last for 50ms but the model run interval is around 200ms
            # which is, by the way, longer than the prediction delay, which is 100ms
            other_values = np.r_[values[-1]['acc'], 100]
        else:
            print('no values')
            other_values = np.r_[0, 0, 0, 100]

        
        out = ml_model([arr[None, :], other_values[None, :]/10])[0]
        out *= 100

        out = out.numpy()

        return out[0], out[1]
    return model

ml = ComponentStarter(
    ImageMLControllerV4, 
    manager, 
    init_kwargs=dict(
        func = get_model, 
        name='ImageMLControllerV4'
    )
)

In [4]:
logger_starter = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    ),
    loop_intervals={'step': 1/100},
)

two_wheel_starter = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_starter = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_starter = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)

bt_ser_out, bt_ser = start_bluetooth_server_v2(manager)

picamera_starter = ComponentStarter(
    Picamera2V2, 
    manager, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_intervals={'step': 1/30},
    sample_setup_kwargs=dict(default_values=[np.zeros((64, 114, 3), dtype=np.uint8)])
)

Starting Serial Port Profile...


NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 89)


In [5]:

two_wheel_starter.register_outgoing_rpc(
    dict(log=logger_starter.incoming_rpcs['log'])
)

angular_speed_control_starter.register_outgoing_rpc(
    dict(
        log=logger_starter.incoming_rpcs['log'],
        ml_inputs = ml.incoming_rpcs['pass_values']
        )
)


bluetooth_control_starter.register_outgoing_rpc(
    dict(log=logger_starter.incoming_rpcs['log'])
)

picamera_starter.register_outgoing_rpc(
    dict(
        log=logger_starter.incoming_rpcs['log'],
        setup_video_saver=logger_starter.incoming_rpcs['setup_video_saver'],
        save_video_frame=logger_starter.incoming_rpcs['save_video_frame'],
        notify_ml = ml.incoming_rpcs['run_model']
        )
)


def get_self_drive_switch():
    return bool(bt_ser_out[0]().get('start'))

to_2wheels = get_switches(ml.outgoing_samples, angular_speed_control_starter.outgoing_samples, get_self_drive_switch)

two_wheel_starter.register_incoming_samples(
    to_2wheels
)

angular_speed_control_starter.register_incoming_samples(
    bluetooth_control_starter.outgoing_samples
)


bluetooth_control_starter.register_incoming_samples(
    bt_ser_out
)

ml.register_outgoing_rpc(
    dict(        
        log=logger_starter.incoming_rpcs['log'],)
)

ml.register_incoming_samples(
    picamera_starter.outgoing_samples
)

In [9]:
ml.start()
bluetooth_control_starter.start()
logger_starter.start()
two_wheel_starter.start()
angular_speed_control_starter.start()
picamera_starter.start()

AssertionError: already started

In [7]:
ml.incoming_rpcs['run_model'].call_semaphore

<Semaphore(value=54)>

In [23]:
ml.incoming_rpcs['pass_values'].call_semaphore

<Semaphore(value=0)>

In [10]:

mon_samples(ml.outgoing_samples)

KeyboardInterrupt: 

In [34]:
ml.outgoing_samples

[<function components.syncronisation.shared_value_v2.<locals>.reader()>,
 <function components.syncronisation.shared_value_v2.<locals>.reader()>]

In [18]:
bt_ser.terminate()
bluetooth_control_starter.process.terminate()
two_wheel_starter.process.terminate()
angular_speed_control_starter.process.terminate()
picamera_starter.process.terminate()
ml.process.terminate()
#logger_starter.process.terminate()


In [8]:
import plotly.express as px
px.imshow(picamera_starter.outgoing_samples[0]())

In [18]:
picamera_starter.outgoing_samples[0]().dtype

dtype('int64')

# playground

In [8]:
l = logger_starter.incoming_rpcs['get_logger'].call('TwoWheelsV3')

In [10]:
l()

KeyboardInterrupt: 

In [3]:

import time
from typing import Callable
class Timer:
    timelapsed: float
    def __init__(self, time_func: Callable[..., float] = time.perf_counter):
        self.time_func = time_func
        
    def __enter__(self):
        self.t0 = self.time_func()
        return self

    def __exit__(self, *exc):
        self.timelapsed = self.time_func()-self.t0
        return False

with Timer() as timer:
    1+1

timer.timelapsed

7.778000053804135e-06

In [40]:

p = time.perf_counter
t0 = p()
1+1
t1 = p()
t1-t0

0.00018729500015979283